In [1]:
import numpy as np
import matplotlib.pyplot as plt

from PIL import Image
from utils import tile_raster_images

In [2]:
import warnings; warnings.filterwarnings('ignore')
import tensorflow as tf

from tensorflow.examples.tutorials import mnist

## Load Datasets

In [3]:
mnist_data = mnist.input_data.read_data_sets('./datasets/mnist/', one_hot=True, seed=90)

## 1. Classify MNIST using Simple Perceptron Model

In [4]:
num_features = 784 # input is 28 width x 28 height matrix
num_labels = 10 # output is 10 digit numbers

## Create Interactive Session

In [5]:
sess = tf.InteractiveSession()

## Set Placeholders

In [6]:
X = tf.placeholder(tf.float32, shape=[None, num_features]) # None indicates batch size
y = tf.placeholder(tf.float32, shape=[None, num_labels])

## Set Variable

In [7]:
W = tf.Variable(tf.zeros([784, 10], dtype=tf.float32, name='weights'))
b = tf.Variable(tf.zeros([10], dtype=tf.float32, name='bias'))

## Set Loss Function

In [8]:
softmax_op = tf.nn.softmax(tf.matmul(X, W) + b)

In [9]:
loss_op = tf.reduce_mean(-tf.reduce_sum(y * tf.log(softmax_op), reduction_indices=[1]))

## Set Optimizer

In [10]:
learning_rate = 0.5

In [11]:
optimizer_op = tf.train.GradientDescentOptimizer(learning_rate).minimize(loss_op)

## Train The Graph

In [12]:
init_op = tf.global_variables_initializer()

correct_prediction_op = tf.equal(tf.argmax(softmax_op, 1), tf.argmax(y, 1))
accuracy_op = tf.reduce_mean(tf.cast(correct_prediction_op, tf.float32))

In [13]:
sess.run(init_op) # run the init_op using an interactive session

In [14]:
batch_size = 64
num_epochs = 1000

In [15]:
for i_epoch in range(1, num_epochs+1):
    
    train_batch = mnist_data.train.next_batch(batch_size)
    if i_epoch % 100 == 0:
        train_accuracy = accuracy_op.eval(feed_dict={X: train_batch[0], y: train_batch[1]})
        print(f'Epoch: {i_epoch}, Training accuracy: {train_accuracy:.4f}')
    
    optimizer_op.run(feed_dict={X: train_batch[0], y: train_batch[1]})

Epoch: 100, Training accuracy: 0.9219
Epoch: 200, Training accuracy: 0.8438
Epoch: 300, Training accuracy: 0.8750
Epoch: 400, Training accuracy: 0.8438
Epoch: 500, Training accuracy: 0.8594
Epoch: 600, Training accuracy: 0.8594
Epoch: 700, Training accuracy: 0.9062
Epoch: 800, Training accuracy: 0.8906
Epoch: 900, Training accuracy: 0.9219
Epoch: 1000, Training accuracy: 0.8906


## Test The Model

In [16]:
accuracy = accuracy_op.eval(feed_dict={X: mnist_data.test.images, y: mnist_data.test.labels}) * 100
print(f'The final accuracy for the simple perceptron model is: {accuracy:.2f}%')

The final accuracy for the simple perceptron model is: 91.31%


## 2. Classify MNIST using Deep Learning Model

In [17]:
width = 28
height = 28
flatten = width * height
num_channels = 1
num_labels = 10

## Create Interactive Session

In [18]:
sess = tf.InteractiveSession()

## Set Placeholders

In [19]:
X = tf.placeholder(tf.float32, shape=[None, flatten])
y = tf.placeholder(tf.float32, shape=[None, num_labels])

In [20]:
X_image = tf.reshape(X, [-1, width, height, num_channels]) # -1 indicates batch size

## Set Variable

#### Convolutional Layer 1

In [21]:
W_conv1 = tf.Variable(tf.truncated_normal([5, 5, 1, 32], stddev=0.1, name='weights_conv1')) # 5x5 filter size
b_conv1 = tf.Variable(tf.constant(0.1, shape=[32], name='bias_conv1')) # need 32 biases for 32 outputs

conv1_layer = tf.nn.conv2d(X_image, W_conv1, strides=[1, 1, 1, 1], padding='SAME') + b_conv1
conv1_layer = tf.nn.relu(conv1_layer)

conv1_map = tf.nn.max_pool(conv1_layer, ksize=[1, 2, 2, 1], strides=[1, 2, 2, 1], padding='SAME')

#### Convolutional Layer 2

In [22]:
W_conv2 = tf.Variable(tf.truncated_normal([5, 5, 32, 64], stddev=0.1, name='weights_conv2')) # 5x5 filter size
b_conv2 = tf.Variable(tf.constant(0.1, shape=[64], name='bias_conv2')) # need 64 biases for 64 outputs

conv2_layer = tf.nn.conv2d(conv1_map, W_conv2, strides=[1, 1, 1, 1], padding='SAME') + b_conv2
conv2_layer = tf.nn.relu(conv2_layer)

conv2_map = tf.nn.max_pool(conv2_layer, ksize=[1, 2, 2, 1], strides=[1, 2, 2, 1], padding='SAME')

#### Fully Connected Layer 1

In [23]:
# flattening last convolution layer output
flatten_matrix = tf.reshape(conv2_map, [-1, 7 * 7 * 64])

In [24]:
W_fc1 = tf.Variable(tf.truncated_normal([7 * 7 * 64, 1024], stddev=0.1))
b_fc1 = tf.Variable(tf.constant(0.1, shape=[1024])) # need 1024 biases for 1024 outputs

fc1_layer = tf.matmul(flatten_matrix, W_fc1) + b_fc1
fc1_layer = tf.nn.relu(fc1_layer)

# dropout layer for reducing overfitting
keep_prob = tf.placeholder(tf.float32)
fc1_layer = tf.nn.dropout(fc1_layer, keep_prob)

#### Fully Connected Layer 2

In [25]:
W_fc2 = tf.Variable(tf.truncated_normal([1024, num_labels], stddev=0.1))
b_fc2 = tf.Variable(tf.constant(0.1, shape=[num_labels]))

fc2_layer = tf.matmul(fc1_layer, W_fc2) + b_fc2

#### Softmax Layer

In [26]:
outputs = tf.nn.softmax(fc2_layer)

## Set Loss Function

In [27]:
loss_op = tf.reduce_mean(-tf.reduce_sum(y * tf.log(outputs), reduction_indices=[1]))

## Set Optimizer

In [28]:
learning_rate = 1e-4

In [29]:
optimizer_op = tf.train.AdamOptimizer(learning_rate).minimize(loss_op)

## Train The Graph

In [30]:
init_op = tf.global_variables_initializer()

In [31]:
sess.run(init_op) # run the init_op using an interactive session

correct_prediction_op = tf.equal(tf.argmax(outputs, 1), tf.argmax(y, 1))
accuracy_op = tf.reduce_mean(tf.cast(correct_prediction_op, tf.float32))

In [32]:
batch_size = 64
num_epochs = 1000

In [33]:
for i_epoch in range(1, num_epochs+1):
    
    train_batch = mnist_data.train.next_batch(batch_size)
    if i_epoch % 100 == 0:
        train_accuracy = accuracy_op.eval(feed_dict={X: train_batch[0], y: train_batch[1], keep_prob: 1.0})
        print(f'Epoch: {i_epoch}, Training accuracy: {train_accuracy:.4f}')
    
    optimizer_op.run(feed_dict={X: train_batch[0], y: train_batch[1], keep_prob: 0.5})

Epoch: 100, Training accuracy: 0.8125
Epoch: 200, Training accuracy: 0.9062
Epoch: 300, Training accuracy: 0.8906
Epoch: 400, Training accuracy: 0.9688
Epoch: 500, Training accuracy: 0.9062
Epoch: 600, Training accuracy: 0.9844
Epoch: 700, Training accuracy: 0.9688
Epoch: 800, Training accuracy: 0.9531
Epoch: 900, Training accuracy: 0.9688
Epoch: 1000, Training accuracy: 0.9844


## Test The Model

In [34]:
# evaluate in batches to avoid out-of-memory issues
num_batches = mnist_data.test.images.shape[0] // batch_size
cummulative_accuarcy = 0.0

In [35]:
for index in range(num_batches):
    
    test_batch = mnist_data.test.next_batch(batch_size)
    cummulative_accuarcy += accuracy_op.eval(feed_dict={X: test_batch[0], y: test_batch[1], keep_prob: 1.0})

print(f'The final accuracy for the CNN model is: {(cummulative_accuarcy/ num_batches)*100:.2f}%')

The final accuracy for the CNN model is: 96.52%


---